# Face Detection Analysis

On this notebook the important elements to analyse are:
* **Detection time reltaive to frame rate and area:** The frame size and the total duration of the video impact on the time spent detecting. We can change the resolution but not the length of the video, instead we can change the frequency the detection happens.
* **Face area relative to frame scale and frame size threshold:** Changing the resolution of the video can impact on the number of faces we detect.
* **Content delta distribution to select threshold:** For the next stage of the pipeline a threshold is required for shot transitions.

In [1]:
DETECTION_PATH = r'\\Desktop-0331tte\d\chiletv-testdata\detection'
DETECTION_CSV = r'..\data\face_detection_testdata.csv'
DETECTION_COLUMNS = [
    'video_id', 'frame_rate', 'batch_size', 'min_face_size', 'max_frame_size', 'frame_scale', 'width', 'height',
    'video_length', 'detection_length']

In [2]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def read_data(detection_path, detection_columns):
    for file in Path(detection_path).glob(r'**/*.detections.json'):
        with file.open('r', encoding='utf8') as fp:
            obj = json.load(fp)
            obj['video_id'] = file.name[:file.name.index('.')]
            if 'frame_scale' not in obj:
                obj['frame_scale'] = float(file.parent.name[1:file.parent.name.index('-')])
            yield {key: obj[key] for key in detection_columns}

csv_file = Path(DETECTION_CSV)
if not csv_file.exists():
    df = pd.DataFrame(read_data(DETECTION_PATH, DETECTION_COLUMNS))
    df.to_csv(csv_file, index=False)
else:
    df = pd.read_csv(csv_file)
df.head()